# Lab 5 - Multi-glitch

**SUMMARY:** *In this lab, we'll modify our original glitch loop to prevent glitched data from leaking from our target. After implementing these changes, we'll take a look at how we can use the Husky's new multi-glitch feature to break past the countermeasures*

**LEARNING OUTCOMES:**

* Implementing countermeasures in firmware
* Differentiating between partial success and normal glitch results
* Using ChipWhisperer-Husky's multi-glitch feature

## Modifying Firmware

In this lab, we'll be revisiting the `simpleserial-glitch` firmware from the first glitch lab we did. This code is pretty simple, so it's easy to modify. It's also fast, which means we can quickly try different glitch settings on it. 

Let's pretend that this loop calculation is actually using sensitive data and that, if this calculation is glitched, the sensitive data will leak. We'll also assume that the end value of the calculation can be verified, but that the firmware can't just send it out instead of doing the calculation. Sound familiar?

Start by opening up `chipwhisperer/hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch.c`. The change we're going to make to it is very simple: if we detect an incorrect calculation, we'll send out a series of 0's instead of the glitched value:

```C
if SS_VER == SS_VER_2_1
uint8_t glitch_loop(uint8_t cmd, uint8_t scmd, uint8_t len, uint8_t* in)
#else
uint8_t glitch_loop(uint8_t* in, uint8_t len)
#endif
{
    volatile uint16_t i, j;
    volatile uint32_t cnt;
    cnt = 0;
    trigger_high();
    for(i=0; i<50; i++){
        for(j=0; j<50; j++){
            cnt++;
        }
    }
    trigger_low();

    if (cnt == 2500) {
        simpleserial_put('r', 4, (uint8_t*)&cnt);
        return 0x00;
    } else {
        uint32_t fallback = 0; //avoid using same variable, just in case
        simpleserial_put('r', 4, (uint8_t*)&fallback);
        return 0x10;
    }
    return 0x10;
}
```

Now, the firmware should avoid sending out glitched values. As you might've guessed, our goal for this lab will be to glitch both the loop and the subsequent check so that we still get the corrupted value back.

Let's connect and program as usual:

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
SS_VER = 'SS_VER_2_1'

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
fw_path = "../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=="SS_VER_2_1":
    target.reset_comms()

Make sure your code still works as normal without glitching:

In [ ]:
# Code Block 1
scope.arm()
target.simpleserial_write("g", [])
scope.capture()

val = target.simpleserial_read('r', 4)
print(val)

Next, we'll do all our normal glitch stuff:

In [ ]:
def reboot_flush():
    reset_target(scope)
    target.flush()

In [ ]:
scope.vglitch_setup('hp', False)

One thing that you might've noticed about the firmware is that you can still tell when the loop is being glitched, even if we don't actually get the glitched value back. This is because the device still sends back an error code and a series of zeros. In fact, it's actually really hard for the target to hide this information - even if it redoes the calculation, we can still tell we got a glitch because it'll take longer for us to receive the result. We could also look at the power trace and see that the device is doing two calculations instead of one.

This is very valuable information, as we can search for successful settings to glitch the loop before trying to find settings to glitch the verification.

We'll setup the glitch controller as normal, but we'll add a new `"partial_success"` group to record when we glitch the loop but not past the verification.

In [ ]:
import chipwhisperer as cw
gc = cw.GlitchController(groups=["partial_success", "success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None, "partial_success":"xy"}, x_index="width", y_index="offset")

Next, fill in settings to glitch the loop:

In [ ]:
gc.set_range("ext_offset", 0, 100)
gc.set_range("width", ???, ???)
gc.set_range("offset", ???, ???)

gc.set_global_step([???]) # reduce to fine tune glitching
gc.set_step("ext_offset", 1)

And write your loop to glitch the loop:

In [ ]:
# ###################
# Add your code here to glitch the loop calculation (Code Block 1)
# ###################
raise NotImplementedError("Add your code here, and delete this.")

Good job! Let's see if we can glitch the verfication as well. We'll begin by setting up the multi-glitch part of the glitch module:

In [ ]:
scope.glitch.num_glitches = 2

And that's it! `ext_offset` and `repeat` are now arrays:

In [ ]:
print(scope.glitch.ext_offset)
print(scope.glitch.repeat)

`width`/`offset` are the same for each glitch, so we don't have to worry about that. We'll keep repeat at 1, so no need to worry about that as well. We'll also keep the first `ext_offset` value constant, so this is almost the same as doing a single glitch. You can see how to set `ext_offset` below. **Note that the second ext_offset is an offset from the first glitch, so `[100, 50]` will put glitches at an `ext_offset` of 100 and 150.**

In [ ]:
scope.glitch.ext_offset = [scope.glitch.ext_offset[0], 1]
scope.glitch.repeat = 1 # same as [1, 1]

Note that glitching past the verification will be a lot harder than glitching the loop since it's a precise glitch far past the trigger. Obviously you want to minimize the `ext_offset` range that you have to search, so you'll have to be smart about what `ext_offset` range you pick. If you're stuck, we've got three hint notebooks you can use to help estimate the range your need for this glitch, with hint 1 being the most challenging and hint 3 being the least. We'll also tell you that you can easily disable and reenable the glitch via `scope.io.glitch_hp = False` and `scope.io.glitch_hp = True`.

The first `ext_offset` is fixed, so we'll reuse the `ext_offset` parameter for our second glitch:

In [ ]:
gc.set_range("ext_offset", ???, ???)

In [ ]:
# ###################
# Add your code here to glitch the loop calculation and the verification (Code Block 2)
# ###################
raise NotImplementedError("Add your code here, and delete this.")